# **Clasificación- Selección de Modelo** 

# **SET UP**

CONEXIÓN CON GOOGLE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **CONFIGURACIÓN DEL ENTORNO**

In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import nltk #Procesamiento de Lenguaje Natural
from nltk.corpus import stopwords
# Corpus de nltk para 'tokenizer', 'stopwords' y 'words' (diccionarios)
# ==============================================================================
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
#Stemming y Lematización
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# Data

In [ ]:
df = pd.read_csv('/content/df_etiquetado.csv',  encoding='utf-8')
df=df.dropna()

In [ ]:
df.head()

corpus_Contenido  Dominant_Topic
0  tardesl escribir presente correo notificarl pr...               4
1  pago factura ver opcion movimiento pago 32250 ...               2
2  ingenicorp 6 solo contacto correo whatsapp lin...               1
3  escribir presente correo notificarl problema i...               4
4  consulta resuelto mayorio cosa indicar hacer a...               4

In [ ]:
len(df)

5811

# Limpieza

In [ ]:
#Importar Diccionario nltk.corpus
stop_words_nltk = list(stopwords.words('spanish'))
#Adición de stopwords al diccionario (nombre de la empresa y demás nombres que no aportan)
stop_words_nltk.extend(('sac','contacto','cordial','cordiales','quedo','quedamos','atento','atentos','favor','rv','image','logo','png','2022','2023','facturedo','da','re','buenas','buenos','hola','status','m','dias','tardes','noches','saludos','saludo','gracias','si','id','escribi','re', 'fw', 'fwd', 'http', 'https', 'www', 'com', 'es', 'para', 'de', 'en', 'y', 'o', 'u', 'que', 'del', 'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas', 'con', 'sin'))
# Palabras sin sentido
stop_words_nltk.extend(('ing','image','quot', 'div','style','sansserif','helvetica','arial', 'span', 'roboto', 'neue','ui', 'segoe', 'applesystem', 'blinkmacsystemfont', 'fontfamily', 'br','td','dir','asunto'))
# Stopwords de inglés
stop_words_nltk.extend(list(stopwords.words('english')))

# Vectorización

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
corpus = df['corpus_Contenido'].values.tolist()
vectorizer = TfidfVectorizer(max_features=5000, stop_words= stop_words_nltk)
X = vectorizer.fit_transform(corpus)

# Partición Train, Validation, Test

In [ ]:
from scipy.sparse import csr_matrix

# Convert csr_matrix to numpy array
matrix_dense = X.toarray()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report

In [ ]:
# Extract the target column from the DataFrame
y = df['Dominant_Topic'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Split the data into new training and Validation sets
X_train1, X_valid, y_train1, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:

# Create an instance of KFold with the desired number of folds
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create an instance of the random forest classifier
rf_model = RandomForestClassifier()

## Optimizar Hiperparametros

In [ ]:
# Define the parameter grid for hyperparameter search
param_grid = {
    'n_estimators': [75, 100, 100],
    'min_samples_leaf': [5, 10, 15]}

# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator= rf_model, param_grid= param_grid, cv=kfold)

# Fit the GridSearchCV instance to the data
grid_search.fit(X_train1, y_train1)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'min_samples_leaf': [5, 10, 15],
                         'n_estimators': [75, 100, 100]})

In [ ]:
# Access the best hyperparameters
best_params = grid_search.best_params_
print(best_params)
# Access the best model
best_model = grid_search.best_estimator_

{'min_samples_leaf': 5, 'n_estimators': 100}


Validation

In [ ]:
# Make predictions on the test data
# Make predictions on the test data
y_pred = best_model.predict(X_train1)
print('Random Forest TRAIN: \n')
# Measure the accuracy of the model
print(classification_report(y_train1, y_pred), '\n')

y_pred = best_model.predict(X_valid)
print('Random Forest VALID: \n')
# Measure the accuracy of the model
print(classification_report(y_valid, y_pred))

Random Forest TRAIN: 

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1017
           1       0.97      0.79      0.87       338
           2       0.99      0.66      0.79       256
           3       1.00      0.68      0.81       103
           4       0.86      0.98      0.92      1381
           5       1.00      0.72      0.83       158

    accuracy                           0.91      3253
   macro avg       0.96      0.80      0.86      3253
weighted avg       0.92      0.91      0.91      3253
 

Random Forest VALID: 

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       424
           1       0.97      0.74      0.84       122
           2       0.95      0.61      0.74       104
           3       1.00      0.67      0.80        36
           4       0.83      0.96      0.89       629
           5       0.98      0.61      0.75        80

    accuracy                 

# **SVM**

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Create an instance of the model
svm_model = SVC()

## Optimizar Hiperparametros

In [ ]:
# Define the parameter grid for hyperparameter search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 'scale'] }

In [ ]:
# Create an instance of GridSearchCV
grid_search = GridSearchCV( estimator= svm_model, param_grid= param_grid, cv= kfold)

# Fit the GridSearchCV instance to the data
grid_search.fit(X_train1, y_train1)

# Access the best hyperparameters
best_params = grid_search.best_params_
print(best_params)
# Access the best model
best_model = grid_search.best_estimator_

{'C': 10, 'gamma': 0.1, 'kernel': 'linear'}


Validation

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_train1)
print('SVM TRAIN: \n')
# Measure the accuracy of the model
print(classification_report(y_train1, y_pred), '\n')

# Make predictions on the test data
y_pred = best_model.predict(X_valid)
print('SVM VALID: \n')
# Measure the accuracy of the model
print(classification_report(y_valid, y_pred))

SVM TRAIN: 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1017
           1       1.00      1.00      1.00       338
           2       1.00      1.00      1.00       256
           3       1.00      0.99      1.00       103
           4       1.00      1.00      1.00      1381
           5       0.99      1.00      1.00       158

    accuracy                           1.00      3253
   macro avg       1.00      1.00      1.00      3253
weighted avg       1.00      1.00      1.00      3253
 

SVM VALID: 

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       424
           1       0.94      0.85      0.89       122
           2       0.93      0.87      0.90       104
           3       1.00      0.86      0.93        36
           4       0.92      0.95      0.93       629
           5       0.97      0.82      0.89        80

    accuracy                           0.93      

# **Logit**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Create an instance of the model
logreg_model = LogisticRegression()

## Optimizar Hiperparametros

In [ ]:
# Define the parameter grid for hyperparameter search
param_grid = {
    'C': [0.01, 0.1,1],
    'penalty': ['l2'] }

In [ ]:
# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator=logreg_model, param_grid=param_grid, cv=kfold)

# Fit the GridSearchCV instance to the data
grid_search.fit(X_train1, y_train1)

# Access the best hyperparameters
best_params = grid_search.best_params_
print(best_params)
# Access the best model
best_model = grid_search.best_estimator_

{'C': 1, 'penalty': 'l2'}


Validation

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_train1)
print('LOGIT TRAIN: \n')
# Measure the accuracy of the model
print(classification_report(y_train1, y_pred), '\n')

# Make predictions on the test data
y_pred = best_model.predict(X_valid)
print('LOGIT VALID: \n')
# Measure the accuracy of the model
print(classification_report(y_valid, y_pred))

LOGIT TRAIN: 

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1017
           1       0.99      0.86      0.92       338
           2       1.00      0.81      0.89       256
           3       1.00      0.66      0.80       103
           4       0.90      0.99      0.95      1381
           5       1.00      0.79      0.88       158

    accuracy                           0.94      3253
   macro avg       0.98      0.85      0.90      3253
weighted avg       0.95      0.94      0.94      3253
 

LOGIT VALID: 

              precision    recall  f1-score   support

           0       0.93      0.94      0.93       424
           1       0.96      0.74      0.83       122
           2       0.99      0.77      0.86       104
           3       1.00      0.61      0.76        36
           4       0.86      0.96      0.91       629
           5       1.00      0.76      0.87        80

    accuracy                           0.90  

# **Entrenar mejor modelo (Logit menos sobreajuste, buen desempeño F1-score)**

# **Logit**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Create an instance of the model
logreg_model = LogisticRegression()

## Optimizar Hiperparametros

In [ ]:
# Define the parameter grid for hyperparameter search
param_grid = {
    'C': [0.01, 0.1,1],
    'penalty': ['l2'] }

In [ ]:
# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator=logreg_model, param_grid=param_grid, cv=kfold)

# Fit the GridSearchCV instance to the data
grid_search.fit(X_train, y_train)

# Access the best hyperparameters
best_params = grid_search.best_params_
print(best_params)
# Access the best model
best_model = grid_search.best_estimator_

{'C': 1, 'penalty': 'l2'}


Validation

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_train)
print('LOGIT TRAIN: \n')
# Measure the accuracy of the model
print(classification_report(y_train, y_pred), '\n')

# Make predictions on the test data
y_pred = best_model.predict(X_test)
print('LOGIT VALID: \n')
# Measure the accuracy of the model
print(classification_report(y_test, y_pred))

LOGIT TRAIN: 

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1441
           1       0.98      0.87      0.92       460
           2       1.00      0.84      0.91       360
           3       1.00      0.68      0.81       139
           4       0.92      0.99      0.95      2010
           5       1.00      0.82      0.90       238

    accuracy                           0.95      4648
   macro avg       0.98      0.87      0.91      4648
weighted avg       0.95      0.95      0.95      4648
 

LOGIT VALID: 

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       371
           1       0.94      0.81      0.87       100
           2       0.96      0.73      0.83       107
           3       1.00      0.43      0.60        28
           4       0.86      0.97      0.91       499
           5       1.00      0.74      0.85        58

    accuracy                           0.90  

# Guardar el DF con los tópicos

In [ ]:
# assume both dataframes have the same index
df_merged = pd.merge(df, df_dominant_topics[['Dominant_Topic', 'Perc_Contribution']], left_index=True, right_index=True, how='right')
df_merged.head()

In [ ]:
df_merged.to_csv('/content/drive/MyDrive/Colab Notebooks/correos_ZIP/df_topics_peru.csv', header = True, index = False)